# XPECTO Competition System - Integration Testing

This notebook tests the integration between the competition system and the epidemic engine, focusing on:
1. Step callbacks functionality
2. Metrics tracking
3. Scoring system validation

In [ ]:
# Add parent directory to path to ensure imports work correctly
import sys
import os
from pathlib import Path

# Add the project root to the path
sys.path.append(str(Path.cwd().parent))

# Import required modules
from src.competition import CompetitionManager
from src.mirage.engines.base import EngineV1
from src.competition.services.simulation_integration import SimulationIntegration
from src.competition.services.competition_service import CompetitionService
from src.competition.data.storage import LocalStorageProvider
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configure pandas to display more columns
pd.set_option('display.max_columns', None)

## 1. Test Step Callbacks

First, we'll test if the engine correctly registers and executes step callbacks.

In [ ]:
# Initialize the engine
engine = EngineV1()

# Create a variable to track callback execution
callback_counter = 0
step_data = []

# Define a test callback function
def test_callback(step, state):
    global callback_counter, step_data
    callback_counter += 1
    step_data.append(step)
    print(f"Callback executed for step {step}")

# Register the callback
engine.register_step_callback(test_callback)

# Run the engine for a few steps
engine.run(steps=10)

# Verify callbacks were executed
print(f"\nCallback was executed {callback_counter} times")
print(f"Steps recorded: {step_data}")

## 2. Initialize Competition Manager and Test Integration

In [ ]:
# Create a test data directory
test_data_dir = "test_data"
os.makedirs(test_data_dir, exist_ok=True)

# Initialize competition manager with a new engine
engine = EngineV1()
competition = CompetitionManager(data_dir=test_data_dir, engine=engine)

# Register test player
player_id = competition.setup_player(name="Test Player")

# Select and display standard scenario
competition.set_scenario("standard")
competition.display_scenario_details()

## 3. Test Metrics Tracking During Simulation

In [ ]:
# Set up the simulation
competition.setup_simulation()

# Define a basic intervention strategy for testing
def test_intervention(engine):
    print("Applying test intervention")
    # Set some basic parameters - these should be adapted to your actual engine API
    if hasattr(engine, 'set_lockdown_level'):
        engine.set_lockdown_level(0.5)
    if hasattr(engine, 'allocate_resources'):
        engine.allocate_resources('healthcare', 100)

# Run simulation and capture results
results = competition.run_simulation(steps=50, interventions=[test_intervention])

In [ ]:
# Check the raw metrics data
if 'raw_metrics' in results and 'metrics_history' in results['raw_metrics']:
    metrics_history = results['raw_metrics']['metrics_history']
    print(f"Collected {len(metrics_history)} data points from simulation")
    
    # Convert to DataFrame for easier analysis
    if metrics_history:
        df = pd.DataFrame(metrics_history)
        print("\nMetrics dataframe head:")
        display(df.head())
        
        # Plot some metrics if available
        if 'infected' in df.columns and 'step' in df.columns:
            plt.figure(figsize=(10, 6))
            plt.plot(df['step'], df['infected'])
            plt.title('Infection Progression')
            plt.xlabel('Step')
            plt.ylabel('Infected Population')
            plt.grid(True)
            plt.show()
    else:
        print("No metrics history found in results")
else:
    print("Raw metrics or metrics history not found in results")

## 4. Test Scoring System

In [ ]:
# Display the score breakdown from the simulation
competition.display_score(results)

# Create a storage provider and competition service for direct testing
storage = LocalStorageProvider(data_dir=test_data_dir)
service = CompetitionService(storage)

# Test with some known values
test_scores = [
    # population_survived, gdp_preserved, infection_control, resource_efficiency, time_to_containment
    (1.0, 1.0, 1.0, 1.0, 1.0),  # Perfect score
    (0.9, 0.8, 0.7, 0.6, 0.5),  # Good score
    (0.5, 0.5, 0.5, 0.5, 0.5),  # Average score
    (0.3, 0.2, 0.1, 0.1, 0.0),  # Poor score
    (0.0, 0.0, 0.0, 0.0, 0.0)   # Worst score
]

print("\nScoring system tests with known values:")
print("-" * 80)
print(f"{'Population':^12} {'GDP':^12} {'Infection':^12} {'Resources':^12} {'Containment':^12} {'Final Score':^12}")
print("-" * 80)

for scores in test_scores:
    pop, gdp, inf, res, time = scores
    final = service.calculate_score(pop, gdp, inf, res, time)
    print(f"{pop:^12.2f} {gdp:^12.2f} {inf:^12.2f} {res:^12.2f} {time:^12.2f} {final:^12.4f}")

In [ ]:
# Test individual score components
test_case = {
    "population": (10000, 1000),  # initial, dead
    "gdp": (1000, 800),  # initial, final
    "infection_rate": 0.25,  # max infection rate
    "resources": 500,  # total resources spent
    "containment": (250, 730)  # containment step, total steps
}

# Calculate individual components
population_survived = service.calculate_population_survived_score(
    test_case["population"][0], test_case["population"][1])
gdp_preserved = service.calculate_gdp_preserved_score(
    test_case["gdp"][0], test_case["gdp"][1])
infection_control = service.calculate_infection_control_score(
    test_case["infection_rate"])
resource_efficiency = service.calculate_resource_efficiency_score(
    population_survived, infection_control, test_case["resources"])
time_to_containment = service.calculate_time_to_containment_score(
    test_case["containment"][0], test_case["containment"][1])

# Calculate final score
final_score = service.calculate_score(
    population_survived, 
    gdp_preserved,
    infection_control,
    resource_efficiency,
    time_to_containment
)

print("\nDetailed component scoring test:")
print(f"Test case: {test_case}")
print(f"Population Survived:   {population_survived:.4f} (weight: 0.30)")
print(f"GDP Preserved:         {gdp_preserved:.4f} (weight: 0.20)")
print(f"Infection Control:     {infection_control:.4f} (weight: 0.20)")
print(f"Resource Efficiency:   {resource_efficiency:.4f} (weight: 0.15)")
print(f"Time to Containment:   {time_to_containment:.4f} (weight: 0.15)")
print(f"Final Score:           {final_score:.4f}")

## 5. Test Player Attempts and Leaderboard

In [ ]:
# Display player attempts
competition.display_player_attempts()

# Run official attempt
competition.toggle_practice_mode(is_practice=False)
official_results = competition.run_simulation(steps=50, interventions=[test_intervention])

# Display updated attempts
competition.display_player_attempts()

# Display leaderboard
competition.display_leaderboard()

## 6. Clean Up Test Data

Uncomment to remove test data directory when finished

In [ ]:
# import shutil
# shutil.rmtree(test_data_dir, ignore_errors=True)
# print(f"Removed test data directory: {test_data_dir}")